In [1]:
from __future__ import  print_function ,division
from builtins import range
import numpy as np

Creating our environment

In [2]:
action_space=('u','d','l','r')
class Grid :
  #our environmetn

  def __init__(self,rows,cols,start):
    self.rows=rows
    self.cols=cols
    self.i=start[0]
    self.j=start[1]

  def set(self,rewards,actions):
    self.rewards=rewards
    self.actions=actions

  def set_state(self,s):
    self.i=s[0]
    self.j=s[1]

  def current_state(self):
    return (self.i,self.j)

  def is_terminal(self,s):
    return s not in  self.actions

  def get_next_state(self,s,a):

    #setting the state
    (i,j)=s[0],s[1]
    if a in self.actions[(i,j)]:
      if a=='u':
        i-=1
      if a=='d':
        i+=1
      if a=='r':
        j+=1
      if a=='l':
        j-=1
    return(i,j)
  def all_states(self):
    return set(self.actions.keys()) | set(self.rewards.keys())




Defining rewards and actions

In [3]:
def standard_grid():
  #. . . 1
  #. x . -1
  #s . . .
  grid=Grid(3,4,(2,0))
  rewards={(0,3):1,(1,3):-1}
  actions={
      (0,0):('d','r'),
      (0,1):('l','r'),
      (0,2):('l','d','r'),
      (1,0):('u','d'),
      (1,2):('u','d','r'),
      (2,0):('u','r'),
      (2,1):('l','r'),
      (2,2):('l','r','u'),
      (2,3):('l','u'),
        }
  grid.set(rewards,actions)
  return grid


  

two function just for displaying values and policy

In [4]:
def print_values(V, g):
    for i in range(g.rows):
        print("---------------------------")
        for j in range(g.cols):
            v = V.get((i, j), 0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")  
        print("")
def print_policy(P,g):
  for i in range(g.rows):
    print('--------------------------------')
    for j in range(g.cols):
      a=P.get((i,j)," ")
      print(" %s    |" %a,end="")
    print("")
  

      


calculating transition probablities and rewards

In [7]:
transition_probs={}
rewards={}
for row in range(grid.rows):
  for col in range(grid.cols):
    state=(row,col)
    if not grid.is_terminal(state):
      for action in action_space:
        next_state=grid.get_next_state(state,action)
        transition_probs[(state,action,next_state)]=1
        if next_state in grid.rewards:
          rewards[(state,action,next_state)]=grid.rewards[next_state]
print(rewards)
print('_____________________')
print(transition_probs)

{((0, 2), 'r', (0, 3)): 1, ((1, 2), 'r', (1, 3)): -1, ((2, 3), 'u', (1, 3)): -1}
_____________________
{((0, 0), 'u', (0, 0)): 1, ((0, 0), 'd', (1, 0)): 1, ((0, 0), 'l', (0, 0)): 1, ((0, 0), 'r', (0, 1)): 1, ((0, 1), 'u', (0, 1)): 1, ((0, 1), 'd', (0, 1)): 1, ((0, 1), 'l', (0, 0)): 1, ((0, 1), 'r', (0, 2)): 1, ((0, 2), 'u', (0, 2)): 1, ((0, 2), 'd', (1, 2)): 1, ((0, 2), 'l', (0, 1)): 1, ((0, 2), 'r', (0, 3)): 1, ((1, 0), 'u', (0, 0)): 1, ((1, 0), 'd', (2, 0)): 1, ((1, 0), 'l', (1, 0)): 1, ((1, 0), 'r', (1, 0)): 1, ((1, 2), 'u', (0, 2)): 1, ((1, 2), 'd', (2, 2)): 1, ((1, 2), 'l', (1, 2)): 1, ((1, 2), 'r', (1, 3)): 1, ((2, 0), 'u', (1, 0)): 1, ((2, 0), 'd', (2, 0)): 1, ((2, 0), 'l', (2, 0)): 1, ((2, 0), 'r', (2, 1)): 1, ((2, 1), 'u', (2, 1)): 1, ((2, 1), 'd', (2, 1)): 1, ((2, 1), 'l', (2, 0)): 1, ((2, 1), 'r', (2, 2)): 1, ((2, 2), 'u', (1, 2)): 1, ((2, 2), 'd', (2, 2)): 1, ((2, 2), 'l', (2, 1)): 1, ((2, 2), 'r', (2, 3)): 1, ((2, 3), 'u', (1, 3)): 1, ((2, 3), 'd', (2, 3)): 1, ((2, 3), 'l'

In [ ]:
Defining a policy evaluation function

In [10]:
def policy_evaluation(grid,policy,V=None):
  iteration=0
  while True:
    for state in grid.all_states():
      if not grid.is_terminal(state):
        new_v=0
        for action in action_space:
          for next_state in grid.all_states():
            #deter,inistic policy
            action_prob=1 if policy.get(state)==action else 0
            r=rewards.get((state,action,next_state),0)
            new_v+=action_prob*transition_probs.get((state,action,next_state),0)*(r+0.9*V[next_state])
        V[state]=new_v
    iteration=iteration+1
    #print_values(V,grid)
    print('\n')
    if iteration==25:
      break
    return V


inintializing with random policy

In [ ]:
#starting with a random policy
grid=standard_grid()
policy={}
for state in grid.actions.keys():
  policy[state]=np.random.choice(action_space)
print_policy(policy,grid)

--------------------------------
 r    | d    | r    |      |
--------------------------------
 u    |      | l    |      |
--------------------------------
 u    | r    | l    | u    |


initializing all values with 0

In [ ]:
# initialize V(s) = 0
V = {}
for s in grid.all_states():
    V[s] = 0

print_values(V, grid)

---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|
---------------------------
 0.00| 0.00| 0.00| 0.00|


Value itertion:we first perform evaluation just one time and we just use second loop of policy iteration approch, we perform value iteration until convergence


In [22]:
V=policy_evaluation(grid,policy,V)
while True:
  is_policy_converged = True
  for state in grid.actions.keys():
    best_value=float('-inf')
    for action in action_space:
      v=0
      for next_state in grid.all_states():
        r=rewards.get((state,action,next_state),0)
        v+=transition_probs.get((state,action,next_state),0)*(r+0.9*V[next_state])
      if v>best_value:
        best_action=action
        best_value=v
    policy[state]=best_action
    if best_action != policy[state]:
            is_policy_converged = False
  itteration+=1
  if is_policy_converged:
    break

print_policy(policy, grid)
print_values(V,grid)




--------------------------------
 r    | r    | r    | g    |
--------------------------------
 u    | x    | u    | h    |
--------------------------------
 u    | r    | u    | l    |
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|
